In [20]:
import lib.io_ as io
import pandas as pd

In [21]:
df = io.load_path_file('random_file_paths.txt', drop_subset = ['recording_MBID', 'artist_MBID'])
df.drop(['release_MBID'], axis = 1, inplace = True)
df.drop_duplicates(['artist_MBID', 'recording_MBID'], inplace = True)
df.rename({'artist_MBID': 'mlhd_artist_mbid', 'recording_MBID': 'mlhd_recording_mbid'}, inplace=True, axis=1)
df.reset_index(inplace=True, drop=True)

df.head()

,timestamp,mlhd_artist_mbid,mlhd_recording_mbid
0,1108412731,f4a31f0a-51dd-4fa7-986d-3095c40c5ed9,1deb956c-5439-4fbb-b026-5adb4330a934
1,1108422818,db999c3f-f243-4a5f-88d6-0c25243b6661,14e9eb4e-155d-46ff-9a83-a8d5e1936c81
2,1108423325,ce58d854-7430-4231-aa44-97f0144b3372,eced9a9b-cd59-40f8-a580-f27094bd8a89
3,1108594566,e6e879c0-3d56-4f12-b3c5-3ce459661a8e,0a8e9fce-b54c-45dd-8081-4aaa654ef4ec
4,1108679759,3c0a0074-4f26-4d3b-b723-a66bf6cc3753,06a5b648-0950-4892-a975-715291a5de6f


In [22]:
%%time
# MB_rec_names = mb.get_recording_name()
# MB_rec_names = pd.read_parquet('warehouse/MB_tables/MB_rec_names.parquet')
# MB_rec_names.set_index('gid', inplace=True)

# MB_rec_redir = mb.get_recording_redirects()
MB_rec_redirects = pd.read_parquet('warehouse/MB_tables/MB_rec_redirects.parquet')
MB_rec_redirects.set_index('old', inplace=True)

# MB_rec_canonical = mb.get_recording_canonical()
MB_rec_canonical = pd.read_parquet('warehouse/MB_tables/MB_rec_canonical.parquet')
MB_rec_canonical.set_index('old', inplace=True)

MB_artist_credit_list = pd.read_parquet('warehouse/MB_tables/MB_artist_credit.parquet')
MB_artist_credit_list.set_index('rec_gid', inplace=True)

CPU times: user 41.1 s, sys: 14.5 s, total: 55.6 s
Wall time: 50.8 s


In [23]:
MB_artist_credit_list.head()

,rec_name,artist_credit
rec_gid,,
ceb174fd-1281-45f4-a0e7-a08f2d2bfc25,Playboy Mommy,Tori Amos
9869027b-0076-42e2-9675-f5bcc3a55fcb,On a Slow Boat to China,Bing Crosby
0bb80970-1548-417f-92e0-3812d752d0d4,If I Knew You Were Comin' I'd've Baked a Cake,Bing Crosby
3277d4d3-784f-4006-b9c3-debee7e0f2f5,Zip A Dee Doo Dah,Bing Crosby
78e3d13c-d52a-4ca6-aec5-0e2cdcf27551,You Gotta Start Off Each Day With a Song,Bing Crosby


In [24]:
def replace(value, lookup_df, col_name):
# applyping function for replacing values.
    try:
        return lookup_df.at[value, col_name]
    except KeyError:
        return value

def replace_multi(value, lookup_df):
    try:
        return tuple(replace(value, lookup_df, col_name) for col_name in lookup_df.columns)
    except KeyError:
        return tuple(None for col_name in lookup_df.columns)

In [28]:
%%time
### mlhd_recording_mbid ###

df['mlhd_canonical_mbid'] = df['mlhd_recording_mbid'].map(lambda x: replace(x, MB_rec_redirects, 'new'))
df['mlhd_canonical_mbid'] = df['mlhd_canonical_mbid'].map(lambda x: replace(x, MB_rec_canonical, 'new'))
# df['recording_name'] = df['mlhd_canonical_mbid'].map(lambda x: replace(x, MB_rec_names, 'name'))

rec_name_artist_credit = df['mlhd_canonical_mbid'].map(lambda x: replace_multi(x, MB_artist_credit_list))
df['rec_name'], df['artist_credit'] = zip(*rec_name_artist_credit)

df.head()

CPU times: user 8.29 s, sys: 4.1 ms, total: 8.29 s
Wall time: 8.3 s


,timestamp,mlhd_artist_mbid,mlhd_recording_mbid,mlhd_canonical_mbid,rec_name,artist_credit
0,1108412731,f4a31f0a-51dd-4fa7-986d-3095c40c5ed9,1deb956c-5439-4fbb-b026-5adb4330a934,1deb956c-5439-4fbb-b026-5adb4330a934,Anywhere,Evanescence
1,1108422818,db999c3f-f243-4a5f-88d6-0c25243b6661,14e9eb4e-155d-46ff-9a83-a8d5e1936c81,b0e80c50-6bca-415c-9515-1b4983329b80,Lady Lady,Mark Joseph
2,1108423325,ce58d854-7430-4231-aa44-97f0144b3372,eced9a9b-cd59-40f8-a580-f27094bd8a89,6ae8e64a-d208-4f4e-8d88-155ed0568344,Building a Mystery,Sarah McLachlan
3,1108594566,e6e879c0-3d56-4f12-b3c5-3ce459661a8e,0a8e9fce-b54c-45dd-8081-4aaa654ef4ec,da1308d1-6037-4ba1-b6fc-a643a4201140,Hallelujah,Jeff Buckley
4,1108679759,3c0a0074-4f26-4d3b-b723-a66bf6cc3753,06a5b648-0950-4892-a975-715291a5de6f,a2a800b4-4969-4bc6-88e9-6928b973890c,Penny & Me,Hanson


In [31]:
df.dropna(subset = ['rec_name', 'artist_credit'], inplace=True)